# Lecture 12: RNN1 - Basics

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import sys

In [2]:
torch.manual_seed(777)  # reproducibility

## (1) Data preparation

In [3]:
idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach "hihell" -> "ihello"
x_data = [[0, 1, 0, 2, 3, 3]] # "hihell"
one_hot_lookup = [[[1, 0, 0, 0, 0],  # 0
                   [0, 1, 0, 0, 0],  # 1
                   [0, 0, 1, 0, 0],  # 2
                   [0, 0, 0, 1, 0],  # 3
                   [0, 0, 0, 0, 1]]] # 4

y_data = [1, 0, 2, 3, 3, 4] # "ihello"
x_one_hot = [[one_hot_lookup[0][x] for x in x_data[0]]]

# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

In [4]:
x_one_hot

[[[1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [1, 0, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [0, 0, 0, 1, 0]]]

## (2) Parameters

In [5]:
num_classes = 5
input_size = 5      # one-hot size
hidden_size = 5     # ouput from the LSTM. 5 to directly predict one-hot
batch_size = 1      # one sentence
sequence_length = 6 # |ihello| == 6
num_layers = 1      # one-layer rnn

## (3) Our model

In [6]:
class Model(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(Model, self).__init__()

        self.num_classes = num_classes
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.sequence_length = sequence_length

        self.rnn = nn.RNN(input_size=self.input_size,
                          hidden_size=self.hidden_size, batch_first=True)
        
    def forward(self, x):
        hidden = self.init_hidden(x)
        
        # Reshape input in (batch_size, sequence_length, input_size)
        x.view(x.size(0), self.sequence_length, self.input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)
        out, hidden = self.rnn(x, hidden)
        out = out.view(-1, num_classes)
        return out

    def init_hidden(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size) for batch_first=True
        return Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))

## (4) Loss & Training

In [7]:
# Instantiate RNN model
model = Model(num_classes, input_size, hidden_size, num_layers)
print(model)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

Model(
  (rnn): RNN(5, 5, batch_first=True)
)


In [8]:
# Train the model
epochs = 100
for epoch in range(1, epochs + 1):
    outputs = model(inputs)
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = "".join([idx2char[c] for c in idx.squeeze()])
    print(f"Epoch: {epoch}, Loss: {loss.data.item()}")
    print(f"Predicted string: {result_str}")

Epoch: 1, Loss: 1.6925097703933716
Predicted string: llllll
Epoch: 2, Loss: 1.522877812385559
Predicted string: llllll
Epoch: 3, Loss: 1.3925014734268188
Predicted string: llllll
Epoch: 4, Loss: 1.2628616094589233
Predicted string: llllll
Epoch: 5, Loss: 1.146023154258728
Predicted string: llllll
Epoch: 6, Loss: 1.0545097589492798
Predicted string: lhelll
Epoch: 7, Loss: 1.0023081302642822
Predicted string: ihelll
Epoch: 8, Loss: 0.9645199179649353
Predicted string: ihelll
Epoch: 9, Loss: 0.9128332138061523
Predicted string: ihelll
Epoch: 10, Loss: 0.878986120223999
Predicted string: ihelll
Epoch: 11, Loss: 0.8401041626930237
Predicted string: ihelll
Epoch: 12, Loss: 0.8045098781585693
Predicted string: ihello
Epoch: 13, Loss: 0.7793033719062805
Predicted string: ihello
Epoch: 14, Loss: 0.7583827972412109
Predicted string: ihello
Epoch: 15, Loss: 0.738359272480011
Predicted string: ihello
Epoch: 16, Loss: 0.717498242855072
Predicted string: ihello
Epoch: 17, Loss: 0.6939412951469421
Pr